## Using XULE to extract XBRL facts  
This interactive environment copies the [free, open-source XULE plugin](https://github.com/xbrlus/xule/) to a working Python environment with [Arelle](https://pypi.org/project/arelle-release/) plus [additional packages](https://raw.githubusercontent.com/xbrlus/xule/jupyter/environment.yml). Click the code cell below (gray background) to bring it in focus, then click the run button above to add and confirm XULE is ready. **The asterisk to the left of the cell indicates the cell is queued/processing**. 

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.30.11
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
temp = plugindir + 'temp/'
tempxule = temp + 'plugin/xule/'
tempxodel = temp + 'plugin/xodel/'
tempserializer = temp + 'plugin/serializer/'
tempSimpleXBRLModel = temp + 'plugin/SimpleXBRLModel/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            os.remove(plugindir + 'semanticHash.py')
            shutil.rmtree(xuledir)
            shutil.rmtree(xodeldir)
            shutil.rmtree(serializerdir)
            shutil.rmtree(SimpleXBRLModeldir)
            shutil.rmtree(temp)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory
!git clone --quiet --depth=1 --branch 30041 --single-branch https://github.com/xbrlus/xule.git temp &> /dev/null
shutil.move(temp + 'plugin/semanticHash.py', plugindir)
shutil.move(tempxule, xuledir)
shutil.move(tempxodel, xodeldir)
shutil.move(tempserializer, serializerdir)
shutil.move(tempSimpleXBRLModel, SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v

Run the cell below to choose ``constants`` used in the XULE expression below.  Click the ``Show code`` link to review/revise this script.

In [ ]:
# @title
def get_user_url(options):
    while True:
        print("\n\nType a number from the list or enter a valid report URI to generate all facts.\nLeave blank to use the first report in the list: ")
        for i, option in enumerate(options):
            print(f"{i + 1}. {option}")
        choice = input("> ")
        # Check if the user entered a number from the list
        try:
            choice = int(choice)
            if 1 <= choice <= len(options):
                return options[choice - 1]
        except ValueError:
            if choice == '':
                return options[0]
            else: 
                pass
        # If the user didn't enter a number, take it as custom input
        return choice

options = ["https://www.sec.gov/Archives/edgar/data/720762/000149315224042532/form10-k.htm",
           "https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm"]

report = get_user_url(options)

print("\nSelect the number of the file format for output.\nLeave blank to use the spreadsheet option: ")
data = {
    "1": "spreadsheet",
    "2": "json"
    }

for key, value in data.items():
    print(f"{key}. {value}")

choice = input("> ")

if choice in data:
    format = data[choice]
else:
    print("No entry - using spreadsheet option.")
    format = "spreadsheet"

location = input("\nEnter a path to the output location or leave the field blank \nto use the default value /content/ (Colab location)> ") or '/content/'

name = input("\nEnter a name for the file without spaces or leave the field blank \nto use the default value 'xbrl-facts'> ") or 'xbrl-facts'

# This deletes the prior version of XULE file if the name matches what  was provided in this routine.
if os.path.exists(name + '.xule'):
    os.remove(name + '.xule')
else: ''

### 1. Define a XULE Expression
The data array created below corresponds to tabs in a spreadsheet for each named section of the presentation linkbase (ie. the financial statements and its disclosures) The function ``.agg-to-dict(1)`` is applied to each different set of 6 characters produced by the list (and printed to the first position) to create section arrays.

Details from the presentation linkbase are printed in each section of the data array (columns 1 -9 in each worksheet). To create a visual hierarchy, an `if/else condition` evaluates each concept's _navigation-depth_ and _preferred-label_ and applies blank spaces preceding the _target.label.text_ string within the statement or disclosure's strucuture.

The code cell below can be edited. Run the cell to write and view the XULE file that will be compiled in the next step. After this XULE cell is run once, re-running the cell above can be used to update ``constants`` in the XULE expression.

In [ ]:
xule_file = '''
/** add constants and namespaces below **/

constant $schema = report('%s')
constant $fileformat = '%s'
constant $filedir = '%s'
constant $filename = '%s'

/** modify rule below **/
output example

output-attribute file-location
output-attribute file-content

{@}

file-content $rule-value
file-location $filelocation

message
"{$rule-value.concept.local-name} {$rule-value.period} {$rule-value} {$rule-value.unit}" 
'''
with open(name + '.xule', mode='w') as file:
    file.write(xule_file % (report, format, location, name))
!cat $name'.xule'

### 2. Run XULE against an XBRL report
The cell below compiles the sample.xule file above (the `{@}` XULE expression returns all facts for an XBRL report) and runs the .zip against the instance document listed in the command (NB: run the first code cell in this notebook with the -h switch instead of -v to get configuration paramters for the command). 

In [ ]:
REPORT_URL = report
FILE_NAME = name + '.xule'
ZIP_NAME = name + '.zip'
FILE_LOCATION = location
LOG_LOCATION = location + 'log.xml'
print('Start compiling XULE.\n\n') 

# compile XULE into .zip 
!arelleCmdLine --plugins "xule" --xule-compile $FILE_NAME \
--xule-rule-set $ZIP_NAME --logFormat="[%(messageCode)s] %(message)s"

# copy XULE and .zip to /content/ directory (a Colab-specific location)
shutil.copy(FILE_NAME, FILE_LOCATION+FILE_NAME)
shutil.copy(ZIP_NAME, FILE_LOCATION+ZIP_NAME)
print('\n\nEnd compiling and start analyzing ' + report)
      
# run .zip to create output
!arelleCmdLine --plugins "xule|transforms/SEC|validate/EFM|inlineXbrlDocumentSet" \
--xule-rule-set $ZIP_NAME \
-v -f $REPORT_URL \
--xule-time .000 --xule-debug --noCertificateCheck \
--httpUserAgent "XULE-Arelle (xbrl.us; support@xbrl.us)" --logFile $LOG_LOCATION


Any **[example]** details listed above were found in the report ('example' is the default name of the output in the sample.xule file above).  Modify and re-run the _1. Define_ and _2. Run_ cells, or change and re-run the report in the _2. Run_ cell to get different results.

